# Apriori
참고: http://blog.naver.com/PostView.nhn?blogId=eqfq1&logNo=221444712369&parentCategoryNo=&categoryNo=45&viewDate=&isShowPopularPosts=true&from=search

In [205]:
import pickle
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

- splitted_movies 리스트 파일로 저장해서 연산 계속 안해도 되도록 해주기

In [290]:
#movies = pd.read_csv('ml-25m/movies.csv')
#ratings = pd.read_csv('ml-25m/ratings.csv')
#movies = pd.read_csv('ml-latest-small/movies.csv')
#ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings = pd.read_csv('ratings_toy.csv')

## user 별로 좋아한 영화 모아주기
- unliked도 함께묶어줌 고치기

In [292]:
ratings['liked'] = np.where(ratings['rating']>=4, 1, 0)
for i in range(len(ratings)):
    if ratings['liked'][i] == 0:
        ratings = ratings.drop([i])
        print(i)

KeyError: 5

In [296]:
ratings['movieId'] = ratings['movieId'].astype('str')
gp_user_like = ratings.groupby(['liked', 'userId'])

splitted_movies = [gp_user_like.get_group(gp)['movieId'].tolist() for gp in gp_user_like.groups]

In [297]:
# 파일에 쓰기
with open('toy_splitted_movies.txt', 'wb') as f:
    pickle.dump(splitted_movies, f)

In [209]:
# 파일 읽어오기
with open('splitted_movies.txt', 'rb') as f:
    movies = pickle.load(f)

### subset => splitted_movies 형태로

In [326]:
movies = pd.read_csv('subset_test.csv')

In [327]:
movies = movies['subset'].tolist()

In [328]:
for i in range(len(movies)):
    movies[i] = movies[i].replace("'", "")
    movies[i] = movies[i].replace("[", "")
    movies[i] = movies[i].replace("]", "")

In [329]:
for i in range(len(movies)):
    movies[i] = movies[i].split(", ")

In [332]:
movies = remove_ones(movies)

In [335]:
movies_train =[]
for i in range(len(movies)):
    movies_train.append(' '.join(movies[i]))

In [336]:
movies_list = list(set(movies_train))

In [339]:
movies_list

[['110', '457'],
 ['1', '110', '296', '457', '588'],
 ['1', '457', '588'],
 ['1', '110', '527'],
 ['50', '296'],
 ['50', '110', '457'],
 ['110', '296', '457', '588'],
 ['1', '50', '296'],
 ['50', '527', '588'],
 ['1', '50', '110', '588'],
 ['1', '50', '296', '527'],
 ['1', '50'],
 ['1', '50', '110', '527'],
 ['50', '457', '527'],
 ['296', '588'],
 ['296', '457'],
 ['110', '296'],
 ['1', '588'],
 ['50', '110', '457', '588'],
 ['1', '110'],
 ['1', '296', '527', '588'],
 ['110', '527', '588'],
 ['1', '110', '457'],
 ['1', '50', '110', '296', '527', '588'],
 ['527', '588'],
 ['1', '296', '457', '527', '588'],
 ['50', '110', '527'],
 ['1', '50', '110', '296', '457', '588'],
 ['296', '457', '527'],
 ['1', '50', '457', '527', '588'],
 ['1', '50', '527', '588'],
 ['50', '588'],
 ['1', '50', '110'],
 ['50', '457', '527', '588'],
 ['1', '296', '457', '588'],
 ['50', '110', '296', '457', '527'],
 ['110', '296', '588'],
 ['110', '296', '457', '527', '588'],
 ['110', '296', '457'],
 ['1', '296', '4

In [338]:
for i in range(len(movies_list)):
    movies_list[i] = movies_list[i].split(" ")

In [340]:
with open('toy_splitted_movies.txt', 'wb') as f:
    pickle.dump(movies_list, f)

## Apriori 계산

In [363]:
te = TransactionEncoder()
te_result = te.fit(movies_list).transform(movies_list)
df = pd.DataFrame(te_result, columns=te.columns_)

In [364]:
df

,1,50,110,296,457,527,588
0,False,False,True,False,True,False,False
1,True,False,True,True,True,False,True
2,True,False,False,False,True,False,True
3,True,False,True,False,False,True,False
4,False,True,False,True,False,False,False
...,...,...,...,...,...,...,...
115,True,True,False,True,False,True,True
116,False,True,True,True,True,False,False
117,False,False,False,False,True,True,True
118,False,True,False,True,True,True,False


### mlxtend의 apriori 함수
- max_len: 아이템셋 최대 길이

In [397]:
frequent_itemsets = apriori(df, min_support=0.1, use_colnames=True)
frequent_itemsets[5:20]

,support,itemsets
5,0.525000,(527)
6,0.525000,(588)
7,0.266667,"(1, 50)"
8,0.266667,"(1, 110)"
9,0.266667,"(296, 1)"
10,0.266667,"(1, 457)"
11,0.266667,"(1, 527)"
12,0.266667,"(1, 588)"
13,0.266667,"(50, 110)"
14,0.266667,"(296, 50)"


In [398]:
for i in range(len(frequent_itemsets['itemsets'])):
    frequent_itemsets['itemsets'][i] = str(frequent_itemsets['itemsets'][i])

/Users/hbae/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [399]:
for i in range(len(frequent_itemsets['itemsets'])):
    frequent_itemsets['itemsets'][i] = frequent_itemsets['itemsets'][i][11:]

/Users/hbae/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [400]:
for i in range(len(frequent_itemsets['itemsets'])):
    frequent_itemsets['itemsets'][i] = frequent_itemsets['itemsets'][i][:-2]

/Users/hbae/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [401]:
# 아이템이 1개 이상인 경우만(슬라이싱 인덱스는 직접 보고 지정해야함)
itemsets = frequent_itemsets.loc[7:]

In [402]:
itemsets = itemsets.reset_index(drop=True)

In [403]:
itemsets

,support,itemsets
0,0.266667,"1, 50"
1,0.266667,"1, 110"
2,0.266667,"296, 1"
3,0.266667,"1, 457"
4,0.266667,"1, 527"
5,0.266667,"1, 588"
6,0.266667,"50, 110"
7,0.266667,"296, 50"
8,0.266667,"457, 50"
9,0.266667,"50, 527"


In [404]:
cand = itemsets['itemsets'].tolist()

In [405]:
for i in range(len(cand)):
    cand[i] = cand[i].replace("'", "")

In [406]:
for i in range(len(cand)):
    cand[i] = cand[i].split(", ")

In [407]:
for i in range(len(cand)):
    for j in range(len(cand[i])):
        cand[i][j] = int(cand[i][j])

In [408]:
for i in range(len(cand)):
    cand[i] = sorted(cand[i])

In [409]:
for i in range(len(itemsets['itemsets'])):
    itemsets['itemsets'][i] = cand[i]

/Users/hbae/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [410]:
itemsets

,support,itemsets
0,0.266667,"[1, 50]"
1,0.266667,"[1, 110]"
2,0.266667,"[1, 296]"
3,0.266667,"[1, 457]"
4,0.266667,"[1, 527]"
5,0.266667,"[1, 588]"
6,0.266667,"[50, 110]"
7,0.266667,"[50, 296]"
8,0.266667,"[50, 457]"
9,0.266667,"[50, 527]"


In [411]:
itemsets.to_csv('toy_apriori_.csv')